In [ ]:
from PIL import Image
from io import BytesIO
import pandas as pd
import openai
import base64
import requests
import io
import math
import html
import re
import cv2
import numpy as np
import imageio
import os
import csv
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


FUNCTION TO FETCH PANORAMIC IMAGES

In [ ]:
# Function to fetch and stitch panoramic images
def fetch_and_stitch_panoramic(lat, lon, api_key, center_heading, file_name, images_dir, heading_step=45, size="450x300", fov="100"):
    # Format location as "lat,lon"
    location = f"{lat},{lon}"

    # Fetch images
    image_files = fetch_streetview_images(location, api_key, center_heading, file_image, images_dir, heading_step, size, fov)

    # Use file_name for the output
    output_file = os.path.join(images_dir, file_name)

    # Stitch images
    stitch_images(image_files, output_file)
    return output_file

# Function to fetch images from Google Street View
def fetch_streetview_images(location, api_key, center_heading, file_image, images_dir, heading_step=45, size="450x300", fov="100"):
    image_filenames = []
    num_images = 360 // heading_step

    # Ensure the output directory exists
    if not os.path.exists(images_dir):
        os.makedirs(images_dir)

    # Define pitch values for ground and sky
    pitches = [10]

    for pitch in pitches:
        # Fetch images from different headings
        for i in range(num_images):
            current_heading = center_heading - (180 - i * heading_step)
            streetview_url = (
                f"https://maps.googleapis.com/maps/api/streetview?"
                f"size={size}&scale=2&location={location}&fov={fov}&heading={current_heading}&pitch={pitch}"
                f"&source=outdoor&mode=driving&key={api_key}"
            )

            response = requests.get(streetview_url)
            if response.status_code == 200:
                image = Image.open(BytesIO(response.content))
                image_filename = f"streetview_{i}_{file_image}"
                image_path = os.path.join(images_dir, image_filename)
                image.save(image_path)
                image_filenames.append(image_path)

            else:
                print(f"Error fetching image for location {location} at heading {current_heading}: {response.status_code}")

    return image_filenames

# Function to stitch images into a panoramic view
def stitch_images(image_files, output_file):
    # Sort images to maintain order
    image_files.sort()

    # Select specific ranges for stitching
    range1 = image_files[1:10]
    selected_files = range1

    # Load images
    images = [cv2.imread(file) for file in selected_files if cv2.imread(file) is not None]

    # Create a stitcher object
    stitcher = cv2.Stitcher_create()
    status, stitched_image = stitcher.stitch(images)

    if status == cv2.Stitcher_OK:
        # Save the stitched image
        cv2.imwrite(output_file, stitched_image)
        print(f"Panoramic image saved as {output_file}")
    else:
        print("Stitching failed")
        if status == cv2.Stitcher_ERR_NEED_MORE_IMGS:
            print("Error: Need more images for stitching")
        elif status == cv2.Stitcher_ERR_HOMOGRAPHY_EST_FAIL:
            print("Error: Homography estimation failed")
        else:
            print(f"Unknown error: {status}")

In [ ]:
from json.decoder import JSONDecodeError
# Define API key and output directory
api_key = "Your_API_key"

# Path to your JSON file
json_file = '/content/dev_visual_environment.json'

# Load the JSON file
with open(json_file, 'r') as file:
    try:
        environment = json.load(file)
    except JSONDecodeError as e:
        print(f"Error loading JSON file: {e}")
        environment = []

# Define the starting index
start_indices = range(600, 800)

for start_index in start_indices:
    print(start_index)
    env = environment[start_index]
    inst = env['Instance_id']  # Extract Instance ID
    print(f"Instance ID: {inst}")

    # Ensure images directory exists
    images_dir = f"/content/drive/MyDrive/map2seq/unseen/panoramic_images/{inst}"
    os.makedirs(images_dir, exist_ok=True)

    # Iterate over locations
    for j, nod in enumerate(env['Locations']):
        # Determine the heading for each node
        if j == 0:  # First node
            heading_pic = nod.get('heading')
        elif j == len(env['Locations']) - 1:  # Last node
            heading_pic = nod.get('heading')
        else:  # Intermediate nodes
            prev_heading = env['Locations'][j - 1].get('heading')
            heading_pic = prev_heading if prev_heading is not None else nod.get('heading')

        if heading_pic is not None:
            lat = nod.get('lat')  # Extract latitude
            lon = nod.get('lon')  # Extract longitude

            # Define the file name for the image
            file_image = f"{inst}_{j}.jpg"

            # Fetch and stitch the panoramic image
            image_filename = fetch_and_stitch_panoramic(
                lat=lat,
                lon=lon,
                api_key=api_key,
                center_heading=heading_pic,
                file_name=file_image,
                images_dir=images_dir
            )

445
Instance ID: 2574
Panoramic image saved as /content/drive/MyDrive/map2seq/unseen/panoramic_images/2574/2574_0.jpg
Panoramic image saved as /content/drive/MyDrive/map2seq/unseen/panoramic_images/2574/2574_1.jpg
Panoramic image saved as /content/drive/MyDrive/map2seq/unseen/panoramic_images/2574/2574_2.jpg
Panoramic image saved as /content/drive/MyDrive/map2seq/unseen/panoramic_images/2574/2574_3.jpg
Panoramic image saved as /content/drive/MyDrive/map2seq/unseen/panoramic_images/2574/2574_4.jpg
Panoramic image saved as /content/drive/MyDrive/map2seq/unseen/panoramic_images/2574/2574_5.jpg
Panoramic image saved as /content/drive/MyDrive/map2seq/unseen/panoramic_images/2574/2574_6.jpg
Panoramic image saved as /content/drive/MyDrive/map2seq/unseen/panoramic_images/2574/2574_7.jpg
458
Instance ID: 272
Panoramic image saved as /content/drive/MyDrive/map2seq/unseen/panoramic_images/272/272_0.jpg
Panoramic image saved as /content/drive/MyDrive/map2seq/unseen/panoramic_images/272/272_1.jpg
P

In [ ]:
def fetch_streetview_images(location_data, api_key, images_dir, size="450x300", fov="120"):
    image_cache = {}  # Cache to store fetched images to avoid duplicates

    # Ensure the output directory exists
    if not os.path.exists(images_dir):
        os.makedirs(images_dir)

    for instance in location_data:
        previous_heading = None  # Reset the previous heading for each new instance

        for idx, loc in enumerate(instance['Locations']):
            lat, lon = loc['lat'], loc['lon']
            file_name = loc['file_name']

            # Determine the heading to use
            if idx == 0:
                heading = loc['heading']  # Use the specified heading for the first image
            else:
                heading = previous_heading  # Use the previous heading for subsequent images

            # Update the previous heading for the next location
            previous_heading = loc['heading']
            coord_key = (lat, lon, heading)  # Unique key for each lat/lon/heading combo

            # Check if we should reuse an image
            if coord_key in image_cache:
                image_cache[coord_key].save(os.path.join(images_dir, file_name))
                print(f"Reused image for {file_name} at ({lat}, {lon}) with heading {heading}")
            else:
                # Build the Street View URL
                streetview_url = (
                    f"https://maps.googleapis.com/maps/api/streetview?"
                    f"size={size}&location={location}&fov={fov}&heading={heading}&pitch={120}"
                    f"&source=outdoor&mode=driving&key={api_key}"
                )
                response = requests.get(streetview_url)

                if response.status_code == 200:
                    # Save the image both to the cache and the target directory
                    image = Image.open(BytesIO(response.content))
                    image_cache[coord_key] = image  # Cache the image
                    image.save(os.path.join(images_dir, file_name))
                    print(f"Fetched and saved image for {file_name} at ({lat}, {lon}) with heading {heading}")
                else:
                    print(f"Error fetching image for {file_name} at ({lat}, {lon}): {response.status_code}")

    print("All images processed.")

# Example usage:
# Load JSON data
with open('/content/dev_visual_environment.json', 'r') as file:
    location_data = json.load(file)

# Define API key and output directory
api_key = "your_API_key"
images_dir = '/content/dev'

# Fetch and save panoramic images
fetch_streetview_images(location_data, api_key, images_dir)


Fetched and saved image for 6918_0.jpeg at (40.720266, -73.997637) with heading 26
Fetched and saved image for 6918_1.jpeg at (40.72047, -73.9975065) with heading 26
Fetched and saved image for 6918_2.jpeg at (40.720674, -73.997376) with heading 26
Fetched and saved image for 6918_3.jpeg at (40.72052033, -73.99694867) with heading 115
Fetched and saved image for 6918_4.jpeg at (40.72036667, -73.99652133) with heading 115
Fetched and saved image for 6918_5.jpeg at (40.720213, -73.996094) with heading 115
Fetched and saved image for 6918_6.jpeg at (40.720544, -73.9959275) with heading 21
Fetched and saved image for 6918_7.jpeg at (40.720875, -73.995761) with heading 21
Fetched and saved image for 6918_8.jpeg at (40.720928, -73.99574) with heading 17
Fetched and saved image for 6918_9.jpeg at (40.7207805, -73.9953565) with heading 117
Fetched and saved image for 6918_10.jpeg at (40.720633, -73.994973) with heading 117
Fetched and saved image for 6918_11.jpeg at (40.720849, -73.994889) wit

KeyboardInterrupt: 

PERFORM THE STRETCHING IMAG FROM STARTING AND ENDING POINT

In [ ]:
# Google API key
api_key = "Your_API_KEY"
location = "40.7250306,-73.9993305"
index = 1
heading = 218
images_dir = '/content/dev'

image_filename = fetch_and_stitch_panoramic(location, api_key=api_key, center_heading = heading, index=1, images_dir=images_dir)

  Panoramic image saved as /content/walking_1/panoramic_image_1.jpg


In [ ]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
def scene_description(image_path, api_key):
    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    }

    payload = {
      "model": "gpt-4o",
      "messages": [
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": "In this panoramic image describe the landmarks visible at this location in NYC."
              "Identify any buildings name, signs, distinctive features, or specific stores."
              " What directional cues (e.g., turns, pathways, junctions) are visible in this scene?"
              "The image is part of the set of images that follow the navigation instruction: "
              "Turn left at the light just ahead of you. Go down the block and turn right at the next light with the Apple Store on the left corner. Continue past the next intersection with Cole Haan on left. At the following light with Happy Socks on the far left corner, cross the street, then stop",
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "low"
              }
            }
          ]
        }
      ],
      "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    response_data = response.json()  # Get the JSON content
    description = response_data['choices'][0]['message']['content']
    return description

In [ ]:
# Set up your OpenAI API key
api_key = 'Your_API_key'

# Load and display the panoramic image (assuming it's a local file)
image_path = "/content/panoramic_image_1 (2).jpg"


graph_scene = scene_description(image_path, api_key)
print(graph_scene)

In the panoramic image of a New York City street, several features and landmarks are visible:

1. **Buildings**: There is a building on the left that has the words "ARONSON ANTIQUE HOUSE" visible, indicating that it may be a retail or showroom location.

2. **Distinctive Features**: The streets are cobblestone, which is characteristic of certain historic neighborhoods in NYC, such as Soho or Greenwich Village.

3. **Pathways**: The image shows a street that curves slightly. This curve might indicate a junction or intersection.

4. **Bicycles and Racks**: On the right side, there is a line of bicycles, likely part of a bike-sharing station.

5. **Trees and Sidewalks**: Trees are planted alongside the sidewalks, contributing to the urban landscape.

In terms of directional cues:

- The street curves slightly, possibly indicating the route to take when turning.
- Given the navigation instructions, the vantage point suggests this location may be along the provided path but prior to an Appl